In [13]:
from gssutils import *

In [14]:
scraper = Scraper('https://www.communities-ni.gov.uk/publications/topic/8182?search=%22Northern+Ireland+Housing+Statistics%22&sort_by=field_published_date')
tabs = { tab.name: tab for tab in scraper.distributions[1].as_databaker() } 

Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored
Cache entry deserialization failed, entry ignored


In [15]:
next_table = pd.DataFrame()

In [16]:
%%capture

%run "T2.1.py"
next_table = pd.concat([next_table, new_table])
%run "T2.2.py"
next_table = pd.concat([next_table, new_table])
%run "T2.3.py"
next_table = pd.concat([next_table, new_table])
%run "T2.4.py"
next_table = pd.concat([next_table, new_table])
%run "T2.5.py"
next_table = pd.concat([next_table, new_table])

In [17]:
next_table['Period'] = next_table['Period'].str.replace('\.0', '')

In [18]:
next_table['NI Household Description'] = next_table['NI Household Description'].apply(pathify)

In [19]:
next_table['NI Household Energy Status'] = next_table['NI Household Energy'].apply(pathify)

In [20]:
next_table['NI Household Energy Status'] = next_table['NI Household Energy Status'].map(lambda cell: cell.replace('/','-'))

In [21]:
next_table['NI Household Energy Status'] = next_table['NI Household Energy Status'].str.lstrip('-')

In [22]:
def user_perc(x):
    
    if (str(x) ==  '*')  | (str(x) == '..'): 
        
        return 'not-defined'
    else:
        return ''
    
next_table['NI Marker'] = next_table.apply(lambda row: user_perc(row['NI Marker']), axis = 1)

In [23]:
next_table = next_table[['Period','NI Household Description','NI Household Energy Status','Unit','Value','Measure Type','NI Marker']]

In [36]:
next_table["Value"][next_table["Value"] == ''] = 0
next_table.head(60)

,Period,NI Household Description,NI Household Energy Status,Unit,Value,Measure Type,NI Marker
0,year/2001,household-central-heating-by-fuel-type,non-central-heating,houses,32170,Count,
1,year/2004,household-central-heating-by-fuel-type,non-central-heating,houses,18300,Count,
2,year/2006,household-central-heating-by-fuel-type,non-central-heating,houses,12780,Count,
3,year/2009,household-central-heating-by-fuel-type,non-central-heating,houses,7460,Count,
4,year/2011,household-central-heating-by-fuel-type,non-central-heating,houses,10530,Count,
5,year/2016,household-central-heating-by-fuel-type,non-central-heating,houses,0,Count,not-defined
6,year/2001,household-central-heating-by-fuel-type,central-heating,houses,615360,Count,
7,year/2004,household-central-heating-by-fuel-type,central-heating,houses,661700,Count,
8,year/2006,household-central-heating-by-fuel-type,central-heating,houses,692220,Count,
9,year/2009,household-central-heating-by-fuel-type,central-heating,houses,732540,Count,


In [15]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
next_table.drop_duplicates().to_csv(out / 'observations.csv', index = False)

In [23]:
scraper.dataset.family = 'affordable-housing'
with open(out / 'observations.csv-metadata.trig', 'wb') as metadata:
    scraper.dataset.license = "http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/"
    metadata.write(scraper.generate_trig())

Northern Ireland Housing Statistics Energy
northern-ireland-housing-statistics-energy


In [24]:
csvw = CSVWMetadata('https://gss-cogs.github.io/family-affordable-housing/reference/')
csvw.create(out / 'observations.csv', out / 'observations.csv-schema.json')